# データの整形

先ほどの学生実験用アンケート結果をダウンロードして Pandas に読み込んでみましょう。

* <a href="https://docs.google.com/spreadsheets/d/1u50kS8Ztmgjjs--S1AM4753quL3QEPRR7xIGjCxcSCw/edit#gid=941271295" target="_blank">学生実験用アンケート結果</a>

    * 左上の「File」→「Download as」→「Comma-separated values (.csv, current sheet)」で保存
    * 保存時の名前は「Questionary - Form Responses 1.csv」のままにしてください。

今回は、「Questionary - Form Responses 1.csv」の書式のままだと非常に使いにくいので、使いやすい形に整形します。以下のコードを実行してください。

In [6]:
import re
seen = []
output = open('./Responses.csv', 'w')
with open('./Questionary - Form Responses 1.csv') as f:
    for i, line in enumerate(f.readlines()):
        if i == 0:
            line = re.sub(r'\..*?\,', ',', line)
            line = re.sub(r'\..*?$', '', line)
            output.write(line)
        else:
            a = line.split(",")
            if a[0] == '':
                continue
            for j in range(len(a)):
                if len(seen) < j + 1:
                    seen.append([])
                if a[j] not in seen[j]:
                    seen[j].append(a[j])
                #print(j, a[j])
            for j in [3, 11, 12, 13, 20]:
                a[j] = seen[j].index(a[j])
            output.write(",".join([str(w) for w in a]))

これで、整形後のデータ Responses.csv が得られて、以降の解析に使えるはずです。

In [7]:
# 数値計算やデータフレーム操作に関するライブラリをインポートする
import numpy as np
import pandas as pd

In [8]:
# URL によるリソースへのアクセスを提供するライブラリをインポートする。
# import urllib # Python 2 の場合
import urllib.request # Python 3 の場合

In [11]:
# データの読み込み
df1 = pd.read_csv('Responses.csv', sep=',') 

In [12]:
df1

,Timestamp,Score,Pen name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,...,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19
0,5/18/2017 13:28:25,NaN,pp,0,50,50,55,60,60.0,4,...,0,0,60,7,6.0,2.80,4.0,20.0,0,2
1,5/18/2017 13:28:37,NaN,chimp,0,70,60,70,70,60.0,4,...,1,0,90,7,5.0,3.20,6.5,6.0,1,1
2,5/18/2017 13:37:00,NaN,あんぱん,0,80,70,65,70,50.0,5,...,0,0,90,7,6.0,2.40,0.0,15.0,1,3
3,5/18/2017 13:40:12,NaN,あかさたな,0,40,50,55,55,50.0,3,...,1,1,30,5,6.0,2.00,10.0,8.0,2,2
4,5/18/2017 13:37:35,NaN,rrr,0,65,60,70,65,50.0,5,...,1,0,120,7,6.0,2.80,12.0,2.0,1,3
5,5/18/2017 13:37:15,NaN,com,0,58,50,60,60,50.0,4,...,1,0,90,7,7.0,2.60,2.0,10.0,2,3
6,5/18/2017 13:37:19,NaN,kk,0,79,60,70,70,60.0,5,...,1,0,100,7,7.0,3.07,4.0,10.0,1,2
7,5/18/2017 13:40:22,NaN,矢澤にこ,0,65,40,75,70,55.0,5,...,1,0,60,3,6.0,2.03,12.0,6.0,2,5
8,5/18/2017 13:37:37,NaN,zxcv,0,65,60,70,70,55.0,4,...,0,1,6,7,6.0,3.30,0.0,3.0,1,2
9,5/18/2017 13:37:39,NaN,あいう,0,65,68,69,65,57.0,5,...,1,0,60,7,5.0,2.80,15.0,10.0,2,3
